In [2]:
import pandas as pd
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\phzxg\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [43]:
data_types = {"author_id": "object", "id": "object"}
df_tweets = pd.read_csv("./tweets_01.csv", index_col=False, header=None, names=['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text', 'stopped_text', 'lem_text'], dtype=data_types, parse_dates=["date"])

In [44]:
df_tweets.shape

(339087, 8)

In [45]:
def clean_text(text):    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions 
    # text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT', '', text) # Removing RT 
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink 
    text = emoji_pattern.sub(r'', text)  # remove emojis 
    text = re.sub('_', '', text)
    return text
df_tweets["cleaned_text02"] = df_tweets["text"].apply(clean_text)

In [46]:
df_tweets = df_tweets[df_tweets["cleaned_text02"].isna()==False]

In [32]:
def select_str(str):
    if "covid" in str.lower() or "pandemic" in str.lower():
        return 1
    return 0
df_tweets["covid"] = df_tweets["cleaned_text02"].apply(select_str)

In [33]:
df_tweets = df_tweets[df_tweets["covid"] == 1]

In [47]:
sid = SentimentIntensityAnalyzer()

In [48]:
df_tweets['scores'] = df_tweets["cleaned_text02"].apply(lambda tweet: sid.polarity_scores(tweet))

In [49]:
df_tweets['compound']  = df_tweets['scores'].apply(lambda score_dict: score_dict['compound'])

In [50]:
def convert_atti(compound):
    if compound < 0:
        return "negative"
    elif compound > 0:
        return "positive"
    else:
        return "neutral"

df_tweets['comp_score'] = df_tweets['compound'].apply(convert_atti) # 0.05?

In [51]:
df_tweets.columns

Index(['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text',
       'stopped_text', 'lem_text', 'cleaned_text02', 'scores', 'compound',
       'comp_score'],
      dtype='object')

In [52]:
df = df_tweets[['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text',
       'stopped_text', 'lem_text', 'scores', 'compound', 'comp_score']]
df.to_csv("./tweets_vader.csv", index=False, header=False)